<a href="https://colab.research.google.com/github/NANNDA3463/ossp_ai2/blob/main/k_fashion_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://kr.object.ncloudstorage.com/aihub-competition/dataset/K-Fashion_Train.zip
!unzip -qq K-Fashion_Train.zip 

--2022-11-29 09:42:31--  https://kr.object.ncloudstorage.com/aihub-competition/dataset/K-Fashion_Train.zip
Resolving kr.object.ncloudstorage.com (kr.object.ncloudstorage.com)... 220.230.123.72
Connecting to kr.object.ncloudstorage.com (kr.object.ncloudstorage.com)|220.230.123.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432747336 (413M) [application/zip]
Saving to: ‘K-Fashion_Train.zip’

K-Fashion_Train.zip 100%[===================>] 412.70M  16.6MB/s    in 26s     

2022-11-29 09:42:59 (15.6 MB/s) - ‘K-Fashion_Train.zip’ saved [432747336/432747336]



In [ ]:
from __future__ import print_function, division

import torch
import torch.utils as utils
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from PIL import Image

from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import pandas as pd
import torchvision
from tqdm import tqdm
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
data_dir = '/content/Train/'
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

image_dataset =  datasets.ImageFolder(data_dir, data_transform)

train_split = 0.9
split_size = int(len(image_dataset) * train_split)
batch_size = 64
num_workers=16

train_set, valid_set = torch.utils.data.random_split(image_dataset, [split_size, len(image_dataset) - split_size])
tr_loader = utils.data.DataLoader(dataset=train_set,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=num_workers)
val_loader = utils.data.DataLoader(dataset=valid_set,
                              batch_size=batch_size,
                              shuffle=False,
                              num_workers=num_workers)
dataloaders = {'train': tr_loader, 'val':val_loader}
dataset_sizes = {}
dataset_sizes['train'] = split_size
dataset_sizes['val'] = len(image_dataset) -split_size 
class_names = image_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=2):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, len(image_dataset.classes))
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 3.1007 Acc: 0.0884
val Loss: 2.9564 Acc: 0.1304

Epoch 1/9
----------
train Loss: 2.8201 Acc: 0.1702
val Loss: 2.7536 Acc: 0.2000

Epoch 2/9
----------
train Loss: 2.6310 Acc: 0.2225
val Loss: 2.6586 Acc: 0.2275

Epoch 3/9
----------
train Loss: 2.5099 Acc: 0.2512
val Loss: 2.6235 Acc: 0.2275

Epoch 4/9
----------
train Loss: 2.4349 Acc: 0.2634
val Loss: 2.5189 Acc: 0.2609

Epoch 5/9
----------
train Loss: 2.3509 Acc: 0.3060
val Loss: 2.4981 Acc: 0.2435

Epoch 6/9
----------
train Loss: 2.3137 Acc: 0.3016
val Loss: 2.4544 Acc: 0.2536

Epoch 7/9
----------
train Loss: 2.2127 Acc: 0.3314
val Loss: 2.4736 Acc: 0.2725

Epoch 8/9
----------
train Loss: 2.2244 Acc: 0.3316
val Loss: 2.4539 Acc: 0.2812

Epoch 9/9
----------
train Loss: 2.2038 Acc: 0.3393
val Loss: 2.4894 Acc: 0.2464

Training complete in 14m 38s
Best val Acc: 0.281159


In [ ]:
!wget https://kr.object.ncloudstorage.com/aihub-competition/dataset/K-Fashion_Test.zip
!unzip -qq K-Fashion_Test.zip 

--2022-11-29 09:57:50--  https://kr.object.ncloudstorage.com/aihub-competition/dataset/K-Fashion_Test.zip
Resolving kr.object.ncloudstorage.com (kr.object.ncloudstorage.com)... 220.230.123.70
Connecting to kr.object.ncloudstorage.com (kr.object.ncloudstorage.com)|220.230.123.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7374788914 (6.9G) [application/zip]
Saving to: ‘K-Fashion_Test.zip’

K-Fashion_Test.zip  100%[===================>]   6.87G  17.6MB/s    in 6m 43s  

2022-11-29 10:04:35 (17.5 MB/s) - ‘K-Fashion_Test.zip’ saved [7374788914/7374788914]



In [ ]:
class TestDataset(utils.data.Dataset):
  def __init__(self, root, transform=None):
    self.root = root
    self.image_list = os.listdir(root)
    self.transform = transform
  
  def __len__(self):
    return len(self.image_list)

  def __getitem__(self, index):
    image_path = os.path.join(self.root, self.image_list[index])
    image = np.array(Image.open(image_path))
    image = self.transform(image)
    return self.image_list[index], image

In [ ]:
test_transform = transforms.Compose([
      transforms.ToPILImage(),
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

In [ ]:
test_dataset = TestDataset('Test',transform=test_transform)
test_dataloader = utils.data.DataLoader(test_dataset, batch_size=256, num_workers=16)

In [ ]:
result = []
for fnames, data in tqdm(test_dataloader):
  data = data.to(device)
  output = model_ft(data)
  _,pred = torch.max(output,1)
  for j in range(len(fnames)):
    result.append(
        {
          'filename':fnames[j],
          'style':pred.cpu().detach().numpy()[j]
        }
    )

100%|██████████| 471/471 [31:09<00:00,  3.97s/it]


In [ ]:
pd.DataFrame(sorted(result,key=lambda x:x['filename'])).to_csv('fashion_submission.csv',index=None)